##### `        -------------------->     Ortho Prediction Software Developing by krish:     <----------------------------`
                   Check all Function Working better or Not
                   main / Global Variable use in this program

In [41]:
import sys
import os                  # For system operation
import glob                #list the file unix system
import subprocess          #to run blastp from OS
import time                # to display time
from alive_progress import alive_bar #For Progress Bar
import multiprocessing  #For Parallel Processing
import queue            # For Multiprocessing
import numpy.matlib     # Matrix operation 
import numpy as np      # For Mathmatical (Algebra) Operation
import copy             # To copy file
import operator   #to concate the list we will remove this later 
import argparse
import datetime         #to display current time & calculate difference
from Bio import SeqIO  # For Bio Python Sequence Object

In [1]:
#Blastp = /usr/bin/blastp
Blastp = "blastp"
Blastp_data =  './blastp_data/'
cpu_count = 1 # Default value
blastp_matrix = "BLOSUM62" 
genome = ""
infinite_loop = 70
inflation_factor = 1.4
#mode = None
mode = None
Cluster_out = "./cluster_out/"
#threshold_score : 0
threshold_score =0      #5 is given in Remark Server / 
Save_raw_blastp_score = False
Score_file = "./score_file"
Species = "./species/"
species = "./species/"
verbose = False
Log_file_name = "krish_ortho_jp"

queryV = None # Only for test Purpose # Always Preferred AAE 
#queryV is created by One way Thr..... Function

###### 1. GetMatrixNumber()

In [2]:
#The Following matrix files must be in Folder
import os
def GetMatrixNumber():
    "This Function will Return the Matrix name choosed by User"
    import sys
    print ("\n1. BLOSUM45\n2. BLOSUM62\n3. BLOSUM82\n4. Quit")

    metrix_num = input("\nEnter a matrix number: ")
    if metrix_num not in ("1","2",'3'):
        print("Wrong input *%s*  Sorry not in the list\n"%metrix_num,"*"*20,"Good Bye","*"*20,"\n")
        sys.exit(1)
    if metrix_num =="1":
        return "BLOSUM45"
    if metrix_num == "2":
        return "BLOSUM62"
    if metrix_num == "3":
        return "BLOSUM82"
#GetMatrixNumber()

###### 2. GetQuerySequence(genome)

In [3]:
#This Function is Developed By Krishna
def get_query_seq(genome):
    try:
        return [str((seq_record.id+seq_record.seq)) for seq_record in SeqIO.parse(genome,"fasta")]
    except IOError as err:
        print(str(err))
        

In [4]:
#This is Old Function
def GetQuerySequence(genome):
    """This Function Read Fastaq Files and return as a list Format with gene Position as a index
    Later I will reduce code Lengths and Speed"""
    global species
    gene_sequence=""
    gene_sequence_list= []
    try:
        with open(species+genome) as gene:
            for each_line in gene:
                if ">" in each_line:
                    if gene_sequence != "":                        
                        gene_sequence_list.append(gene_sequence)
                        gene_sequence = ""
                gene_sequence = gene_sequence+each_line
            gene_sequence_list.append(gene_sequence)                   
        return gene_sequence_list  
    except IOError as err:
        print ("IOError occurred in GetQuerySequence function : " + str(err))

###### 3.Write Query(query, parallel_num)

In [5]:
def WriteQuery(query, parallel_num):
    # This will write query as a sequence and parallel_num is for file name like query_parallel_num
    try:
        with open("./query"+"_"+str(parallel_num), "w") as write_query:
            write_query.write(query)
    except IOError as err:
        print ("IOError occurred in WriteQuery function : " + str(err))

###### 4. RunBlast( Subject , parallel_num)

In [6]:
import subprocess
def RunBlast(subject, parallel_num):
    """By this Function it will create a Pipe line to run Blastp in Computer by input Parameter
     subject is a small size gene passed not a whole
     parallel_nu is the CPU selected by user"""
    subject = Species+subject 
    Blastp = "blastp"
    cmd = [Blastp, "-query","./query"+"_"+str(parallel_num), "-subject", subject,"-matrix", blastp_matrix,
           "-outfmt", "10 qseqid sseqid score length"]
    
    run_blastp =subprocess.Popen(cmd,stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    #qseqid = query (e.g. gene sequence id) , sseqid = subject(e.g. reference genome)
    #sequence id score = Rawscore length = Alignment length
    
    run_blastp_stream = run_blastp.communicate()
    run_blastp_output_stream = run_blastp_stream[0]
    #run_blastp_error_stream = run_blastp_stream[1]  
    
    #display(run_blastp)
    #display(run_blastp_error_stream)
    return run_blastp_output_stream
#RunBlast("AAE","AAE")

In [7]:
#Run Manually 
#!blastp -query query_1 -subject "AAE"

###### 5. Get_Same_Species_Forward_Best_Hit(blastp_score):

In [8]:
def Get_Same_Species_Forward_Best_Hit(blastp_score):
    """ If the User specified the Same input like 1("AAE") & 1 ('AAE') i.e if species_of_query == species_of_subject
    It will Save Best Score without reversing RunBlast
    Search the forward best hit among the blastp scores of same species.
    Because there are an duplicated genes in a same genome.
    When the blastp score compare with blastp score of duplicate gene, if score and length are same, blastp score of duplicated
    gene i sadded to a second best score."""
    #blastp_score is Output File by doing blast
    blastp_score_split_list  = []
    temp_best_score = ["-1",'-1','-1']
    second_temp_best_score = []
    best_score = []
    second_best_score = []
    blastp_score_split = blastp_score.split('\n')
    del blastp_score_split[-1] #delete of [''] in last index 
    for i in blastp_score_split:
        blastp_score_element = i.split(",")
        blastp_score_split_list.append(blastp_score_element)
    for k in blastp_score_split_list: #ex) k in ['gi|15605613|ref|NP_212986.1|', 'gi|15605613|ref|NP_212986.1|', '3702', '699']
        if k[0] == k[1]:
            best_score.append(k)
        elif k[0] !=k[1]:
            print(k[2], temp_best_score[2])
            if int(k[2]) >int(temp_best_score[2]): #Compare the Score , Greater than
                temp_best_score = k
            elif int(k[2]) == int(temp_best_score[2]): # Is equal
                if int(k[3])> int(temp_best_score[3]) : #Compare the length
                    temp_best_score = k
                elif int(k[3]) == int(temp_best_score[3]):
                    second_temp_best_score.append(k)
        #print( "################ temp Best Score #####################", temp_best_score)
        second_best_score.append(temp_best_score)
        for j in second_temp_best_score:
            if j[2] == temp_best_score[2] and j[3] == temp_best_score[3]:
                second_best_score.append(j)
        for m in second_best_score:
            if (best_score[0][2] == m[2] and int(best_score[0][3])<= int(m[3])) or int(best_score[0][2])< int(m[2]): 
                #'104'<"23" is True because of string. So the int Function is Used.
                best_score.append(m)
    return best_score     

###### 6. GetForwardBestHit(blastp_score)

In [78]:
def GetForwardBestHit(blastp_score):
    "Search the forward best hit among blastp scores of same species"
    #print("Running Get Forward Best Hit")
    blastp_score_split_list = []
    temp_best_score = ['-1','-1','-1']
    second_temp_best_score = []
    best_score = []
    blastp_score_split = blastp_score.split("\n")
    del blastp_score_split[-1] #Remove last index of "\n"
    for i in blastp_score_split:
        blastp_score_element = i.split(",")
        blastp_score_split_list.append(blastp_score_element)
    for k in blastp_score_split_list:
        # ex) k is ["gi|15605613|ref|NP_212986.1|",'gi|15605613|ref|NP_212986.1|', '3702', '699']
        if int(k[2])>int(temp_best_score[2]): # Compare Score
            temp_best_score = k
        elif int(k[2]) == int(temp_best_score[2]):
            if int(k[3])> int(temp_best_score[3]): #Comapare the Length
                temp_best_score = k
            elif int(k[3]) == int(temp_best_score[3]):
                second_temp_best_score.append(k)
    #print("## temp best Score -->",temp_best_score)
    best_score.append(temp_best_score)
    for j in second_temp_best_score:
        if j[2] == temp_best_score[2] and j[3] == temp_best_score[3]:
            best_score.append(j)
    return best_score , blastp_score_split_list

###### 7. DivisionParallelQuery(queryV,query_division_value, cpu_count , queryV_len)

In [10]:
def DivisionParallelQuery( queryV ,query_division_value,cpu_count,queryV_len):
    """ Running Division parallel Query,input is species_of_query , species_of_subject, queryV, parallel_num"""
    parallel_query = []
    parallel_query_start = 0
    if queryV_len % cpu_count == 0: #Perfect division by CPU Count
        for i in range(cpu_count):
            i+=1
            if parallel_query_start ==0:
                parallel_query.append(queryV[int(parallel_query_start):i*int(query_division_value)])
                parallel_query_start +=1
            else:
                parallel_query.append(queryV[int(parallel_query_start)*int(query_division_value):i*int(query_division_value)])
                parallel_query_start +=1
    else: # If the division is imperfect
        for i in range(cpu_count):
            i += 1
            if parallel_query_start == 0:
                parallel_query.append(queryV[int(parallel_query_start):i*int(query_division_value)])
                parallel_query_start +=1
            elif i<cpu_count:
                parallel_query.append(queryV[int(parallel_query_start)*int(query_division_value):])
                parallel_query_start +=1
        return parallel_query

###### 8. Run Parallel Query

In [76]:
def RunParallelQuery(species_of_query, species_of_subject, queryV, parallel_num):
    #from progress import bar
    #i --> Species_of_query , k --> Species_of_subject , queryV is Genome list whole file (i , k are part of this)
    """This Function helps to run operation on Several Process
    Run the Following Functions. WriteQuery , RunBlast , Get_Same_Species_Forward_Best_Hit, GetForwardBestHit"""
    #print("Parallel Query Running") 
    #bar = Bar("processing"+str(cpu_count),max = len(queryV))
    #bar = bar("processing"+str(cpu_count),max = len(queryV))  #python 3
    for j in queryV:
        #bar.next()
        WriteQuery(j, cpu_count)
        #WriteQuery() Function Only write file of 1 Gene inside 
        #This Function write the File in same directory with name query_ +(cpu_count). This will later use in blast Purpose
        
        blastp_score= RunBlast(selected_species_dic[species_of_subject],parallel_num)
        #In this Function 2 Parameter slected_species_dic_of_subject[spe..] is whole Genome and
        # parallel_num is for query_"file" like query_1 if cpu-count i.e parallel_num is 1
        
        if blastp_score != "":
            best_score, blastp_score_split_list = GetForwardBestHit(blastp_score.decode())
            #This Function will return the best score from blast Seq and blastp_score_split_list
            #decode() will change to str Format
            if species_of_query == species_of_subject:
                # if Same Input Passed 1 1 , We are testing the Code in this Mode
                # e.g. AAE == AAE It will save best_score without reversing RunBlast.
                same_species_forward_best_score = Get_Same_Species_Forward_Best_Hit(blastp_score.decode())
                
                for best_score_element in same_species_forward_best_score:
                    if best_score_element[0] == best_score_element[1]: #ex [A1 of AAE, A! of AAE , 30 ]
                        with open(Score_file+selected_species_dic[species_of_query]+"_" + selected_species_dic[species_of_subject]+"_oneway_threshold_best_hit_S"+str(threshold_score),"a") as oneway_threshold_best_hit:
                            save_best_score= selected_species_dic[species_of_query]+"_"+best_score_element[0].split("\s")[0]+" "+selected_species_dic[species_of_subject]+ "_"+best_score_element[1].split("\s")[0]+" "+best_score_element[2]+"\n"
                            oneway_threshold_best_hit.write(save_best_score)
                                 #best_score_element[0].split("|" ==> ["gi", '15642790','ref', "NP_227831.1",""])
                    else:
                        # ex ) [ A1 of AAE , A2 of AAE, 30]
                        with open (Score_file+selected_species_dic[species_of_query]+"_"+selected_species_dic[species_of_subject]+
                                  "_second_oneway_threshold_best_hit_S"+ str(threshold_score),"a") as second_onewaythreshold_best_hit:
                                  second_save_best_score = selected_species_dic[species_of_query]+"_"+best_score_element[0].split("\s")[0].split("\s")[0]+\
                                  selected_species_dic[species_of_subject]+"_"+best_score_element[1].split("\s")[0]+" "+best_score_element[2]+" "+best_score_element[3]+"\n"
                                  second_oneway_threshold_best_hit.write(second_save_best_score)
            
                else: # If species_of_query not equal with species_of_subject , run reversing RunBlast
                    for best_score_element in best_score:
                        if not "-1" in best_score_element :
                            with open(Score_file+selected_species_dic[species_of_query]+"_"+selected_species_dic[species_of_subject]+"_S"+str(threshold_score)+"_"+str(parallel_num),"a")\
                            as save_best_hit:
                                best_score_save = selected_species_dic[species_of_query]+"_"+best_score_element[0].split("\s")[0]+" "+ selected_species_dic[species_of_subject]+"_"+\
                                "_"+best_score_element[1].split("\s")[0]+" " +best_score_element[2]+" "+best_score_element[3]+"\n"
                                save_best_hit.write(best_score_save)
                    for blastp_score_split_list_element in blastp_score_split_list :
                        with open(Score_file+selected_species_dic[species_of_query]+"_"+selected_species_dic[species_of_subject]+"_"+"blastp_score_split_list_S"+str(threshold_score)+"_"+str(parallel_num), "a") as save_blastp_score_split_list:
                            blastp_score_split_list_save = selected_species_dic[species_of_query]+"_"+blastp_score_split_list_element[0].split("\s")[0]+" "+selected_species_dic[species_of_subject]+"_"+blastp_score_split_list_element[1].split("\s")[0]+" "+blastp_score_split_list_element[2]+" "+blastp_score_split_list_element[3]+"\n"
                            save_blastp_score_split_list.write(blastp_score_split_list_save)
                
            if save_raw_blastp_score:
                with open(Score_file+selected_species_dic[species_of_query]+"_"+selected_species_dic[species_of_subject]+"_S"+\
                         str(threshold_score)+"_"+str(parallel_num), "a") as save_blastp:
                    save_blastp.write(blastp_score)
                    
            #bar.finish()
            return 

###### 9.Oneway_Threshold_Best_Hit(mode)
modes are 1. Blastp 2. BLASTP using precalculated data 3. Clustering 

In [70]:
#Oneway_Threshold_Best_Hit("1")
#backward_best_hit_work_list.append((1,3,5000))
#max(backward_best_hit_work_list)
#backward_best_hit_work_list
user_selected_number

[1, 2, 3, 4, 5]

In [87]:
backward_best_hit_work_list = [] #For Testing make this Global Variable
def Oneway_Threshold_Best_Hit(mode):
    global queryV,user_selected_number, cpu_count
    "This Function accept the mode and Run Program and return backward_best_hit_work_list"
    process_list = []
    #backward_best_hit_work_list = [] # Container to save backward best hit work # After Testing make this local Variable
    if "1" in mode:
        #print("Blastp Mode is Selected")
        for i in user_selected_number: # user_selected_number is the list of Species 1,2 ..
            queryV =  GetQuerySequence(selected_species_dic[i])
            # Convert to list format with Genome Seq and Genome Id 
            for k in user_selected_number: # Select of Subject
                if k <i: #One Way Blast Method
                    #gene --> query 1->1 1->2 1->3 Forward Checking will skip the same or less
                    print(k)
                    continue
                else:
                    #print("Doing the blastp & Forward best hit searches between %s genome and %s genome"%\
                    #      (selected_species_dic[i],selected_species_dic[k]))
                    queryV_len = len(queryV) #Repeating Time
     
                    if cpu_count ==1:
                        blastp_time_start = time.time() #start Time
                        #print("Running Parallel Query with the Following parameter i,k,cpu_count", i,k,cpu_count)
                        RunParallelQuery(i , k , queryV , cpu_count)
                        blastp_time_end = time.time()
                        #print("The Blastp & Forward best hit searches took %.2f minutes"%((blastp_time_end- blastp_time_start)/60))
                    else:
                        if queryV_len < cpu_count : 
                            # If the number of queryV_len is less than cpu_count, Remark will select the number of queryV_len
                            blastp_time_start = time.time()
                            parallel_query = DivisionParallelQuery(queryV, 1, queryV_len, queryV_len)
                            #1 is query_division_value. Because queryV_len / queryV_len( = cpu_count) is 1
                            for m in range(queryV_len):
                                process = multiprocessing.Process(target = RunParallelQuery, args = (i,k, parallel_query[m],m+1))
                                #args(i => species of query, k = > species of subject , m+1 = > cpu_count ex) 1,2 ...)
                                process_list.append(process)
                                process.start()
                            for n in process_list:
                                n.join()
                            blastp_time_end = time.time()
                            #print("The blastp & forward best hit searches took %.2f minutes"%((blastp_time_end - blastp_time_start)/60))
                        else:
                            blastp_time_start = time.time()
                            query_division_value = queryV_len / cpu_count
                            parallel_query = DivisionParallelQuery(queryV, query_division_value , cpu_count, queryV_len)
                            
                            for m in range(cpu_count):
                                process = multiprocessing.Process(target=RunParallelQuery, args= (i,k,parallel_query[m],m+1))
                                #args(i =>species of query , k = > species of subject , m+1 => cpu_count ex)1,2 ...
                                process_list.append(process)
                                process.start()
                            for n in process_list:
                                n.join()
                            blastp_time_end = time.time()
                            #print("The blastp & forward best hit searches took %.2f minutes"%((blastp_time_end - blastp_time_start)/60))
                            
                    if not i == k:
                        backward_best_hit_work_list.append((i,k,queryV_len))
    elif "2" in mode:
        print("BLASTP using precalculated data mode Running")
        for i in user_selected_number: #Select species to write query
            queryV = GetQuerySequence(selected_species_dic[i])
            for k in user_selected_number: #Select of subject
                if Blastp_data + selected_species_dic[i]+"_"+selected_species_dic[k] + "_oneway_threshold_best_hit_S"+\
                str(threshold_score) in precalculated_data_list:
                    used_precalculated_data_list.append(selected_species_dic[i]+"_"+selected_species_dic[k])
                    continue
                else:
                    if k<i: # gene ===> query 1 -> 1 , 1 ->2  , 1 ->3 ,2 ->2 , 2->3
                        continue
                    else:
                        #print("Doing the blastp & Forward best hit searches between %s genome and %s genome"%(selected_species_dic[i],selected_species_dic[k]))
                        queryV_len = len(queryV)
                                    
                        if cpu_count ==1:
                            blastp_time_start = time.time()
                            RunParallelQuery(i,k,queryV, c)
                            blastp_time_end = time.time()
                            #print("The blastp & forward best hit searches took %.2f minutes"%\
                            #      ((blastp_time_end-blastp_time_start)/60)) 
                        else:
                            if queryV_len<cpu_count:
                                #if the number of queryV_len is less than cpu_count, Remark will select the number of queryV_len.
                                blastp_time_start = time.time()
                                parallel_query = DivisionParallelQuery(queryV,1,queryV_len, queryV_len)
                                #1 is query division_value. Because queryV_len /queryV_len( = cpu_count) is 1
                                for m in range(queryV_len):
                                    process = multiprocessing.Process(target= RunParallelQuery, args = (i,k,parallel_query[m],m+1))
                                    #args (i => species of query, k = > species of subject, m+1 = > cpu_count e.x) 1,2,3...)
                                    process_list.append(process)
                                    process.start()
                                    for n in process_list:
                                        n.join()
                                    blastp_time_end = time.time()
                                    #print("The blastp & forward best hit searches took %.2f minutes"%\
                                    #      ((blsatp_time_end- blastp_time_start)/60))
                                else:
                                    blastp_time_start = time.time()
                                    query_division_value = queryV_len / cpu_count
                                    print("you selected cpu more than1 so query division valueis",\
                                         query_division_value)
                                    parallel_query = DivisionParallelQuery(queryV , query_division_value, cpu_count , queryV_len)
                                    
                                    for m in range(cpu_count):
                                        process = multiprocessing.Process(target = RunParallelQuery, args = (i,k,parallel_query[m],m+1))
                                        #args (i = > species of query, k = > species of subject, m+1 = > cpu_count ex ) 1,2,...
                                        process_list.append(process)
                                        process.start()
                                    for n in process_list:
                                        n.join()
                                    blastp_time_end = time.time()
                                    print("The blastp & forward best hit searches took %.2f minutes"%((blastp_time_end - blastp_time_start)/60))
                            new_calculated_data_list.append(selected_species_dic[i]+"_"+selected_species_dic[k])
                            if not i ==k :
                                backward_best_hit_work_list.append(i,k,queryV_len)
    return backward_best_hit_work_list

In [89]:
#cpu_count = 1#cpu shortage cause broken pipe error if no sufficient core availiable
result = Oneway_Threshold_Best_Hit("1")
#results

384 -1
213 384
97 384
227 384
220 384
200 384
174 384
87 384
78 384
78 384
77 384
73 384
75 384
74 384
72 384
70 384
71 384
69 384
1
202 -1
74 202
60 202
56 202
55 202
54 202
53 202
52 202
53 202
1
2
1
2
3
59 -1
54 59
1
2
3
4
85 -1
79 85
74 85
71 85
72 85
69 85
69 85
67 85
66 85


In [86]:
max(results)

(4, 5, 157)

###### 10. Backward_Best_Hit(args):

In [15]:
def Backward_Best_Hit(args):
    species_of_query, species_of_subject, queryV_len = args
    #Received 3 types of input argument
    start_time_BBH = time.time()
    forward_best_hit_score_list = []
    blastp_score_split_list = []
    print("Start to run the backward best hit between %s genome %genome"%(selected_species_dic[species_of_query],selected_species_dic[species_of_subject]))
    if queryV_len <cpu_count:
        #if the number of queryV_len is less than cpu_count, the cpu_count is changed to queryV_len
        for parallel_num in range(queryV_len):
            parallel_num +=1
            with open(Score_file+selected_species_dic[species_of_query]+"_"+selected_species_dic[species_of_subject]+"_"+"best_score_S"+\
                     str(threshold_score)+"_"+str(parallel_num),"r") as best_hit_score:
                for each_line in best_hit_score:
                    split_each_line = each_line.split(" ")
                    split_each_line[2] = int(split_each_line[2])
                    split_each_line[3] = int(split_each_line[3])
                    print(split_each_line)  #to check
                    forward_best-hit_score_list.append(split_each_line)
            with open(Score_file+selected_species_dic[species_of_query]+"_"+selected_species_dic[species_of_subject]+"_"+"blastp_score_split_list_S"+\
                     str(threshold_score)+"_"+str(parallel_num),"r") as blastp_score:
                for each_line in blastp_score:
                    split_each_line = each_line.split(" ")
                    split_each_line[2] = int(split_each_line[2])
                    split_each_line[3] = int(split_each_line[3])
                    print(split_each_line)
                    blastp_score_split_list.append(split_each_line)
    else:
        for parallel_num in range(cpu_count):
            parallel_num += 1
            with open(Score_file+selected_species_dic[species_of_query]+"_"+selected_species_dic[species_of_subject]+"_"+"best_score_S"+\
                     str(threshold_score)+"_"+str(parallel_num),"r") as best_hit_score:
                for each_line in best_hit_score:
                    split_each_line = each_line.split(" ")
                    split_each_line[2] = int(split_each_line[2])
                    split_each_line[3] = int(split_each_line[3])
                    print(split_each_line)
                    
                    forward_best_hit_score_list.append(split_each_line)
            with open(Score_file+selected_species_dic[species_of_query]+"_"+selected_species_dic[species_of_subject]+"_"+"blastp_score_split_list_S"+\
                     str(threshold_score)+"_"+str(parallel_num),"r") as balstp_score:
                for each_line in blastp_score:
                    split_each_line = each_line.split(" ")
                    split_each_line[2] = int(split_each_line[2])
                    split_each_line[3] = int(split_each_line[3])
                    print(split_each_line)
                    blastp_score_split_list.append(split_each_line)
    #bar = Bar("Searching: "+selected_species_dic[species_of_query]+"_"+selected_species_dic[species_of_subject],max=len(forward_best_hit_score_list))
    
    for forward_best_hit_score_element in forward_best_hit_score_list:
        matching_list = []
        backward_best_score = ["-1","-1",'-1']
        bar.next()
        for element in blastp_score_split:
            if element[1] == forward_best_hit_score_element[1]:
                matching_list.append(element)
            for element in matching_list:
                if int(element[2])> int(backward_best_score[2]):
                    backward_best_score = element
    #        with open('./'+selected_species_dic[species_of_query]+"_"+selected_species_dic[species_of_subject]+'_subtraction'+"_"+str(threshold_score), 'a') as subtraction :
    #            save_data = int(backward_best_score[2]) - int(forward_best_hit_score_element[2])
    #            subtraction.write(str(save_data)+"\n")
    
        if  int(backward_best_score[2])-int(forward_best_hit_score_element[2])<=threshold_score :
            #
            with open(Score_file+selected_species_dic[species_of_query]+"_"+selected_species_dic[species_of_subject]+"_oneway_threshold_best_hit_S"+\
                     str(threshold_score),"a") as other_oneway_threshold_best_hit:
                save_data = forward_best_hit_score_element[0]+" "+forward_best_hit_score_element[1]+" "+str(int(forward_best_hit_score_element[2]))+"\n"
                other_oneway_threshold_best_hit.write(save_data)
    #bar.finish()
    finish_time_BBH = time.time()
    RBH_time = float((finish_time_BBH-start_time_BBH)/60)
    print("BackwardBestHit of %s -%s took %.2f minutes"%(selected_species_dic[species_of_query],selected_species_dic[species_of_subject],RBH_time))
    return RBH_time

###### 11. Search_Equal_BBH_Data(target_A)

In [16]:
def  Search_Equal_BBH_Data(target_A):
    """ Search the equal backward best hit data. ex --> AAE_AAE_backward_best_hit """
    put_data = equal_BBH_data_dic[target_A]
    if put_data[1] ==0:
        pass
    else:
        copy_put_data = copy.copy(put_data) #Copy Module to Copy 
        copy_put_data.insert(0,target_A)
        result.put(copy_put_data)
        equal_BBH_Data_dic[target_A][1] = 0
        print("Put zero in Search_Equal_BBH_Data ----",equal_BBH_data_dic[target_A])
        for i in second_equal_BBH_data:
            if i[2] == 0:
                pass
            else:
                if i[0]==target_A or i[1] == target_A:
                    copy_second_put_data = copy.copy(i)
                    tasks.put(copy_second_put_data)
                    #Don't put results as queue. Because the tasks will put copy_second_put_data to results as queue.
                    i[2] ==0
                    print("--put zero in second_equal_BBH_data--",i)
    return

######  12. Search_Unequal_BBH_Data(target_B):

In [17]:
def Search_Unequal_BBH_Data(target_B):
    """ Search the unequal backward best hit data. ex) AAE_CAC_backward_best_hit"""
    for i in unequal_BBH_data:
        if i[2]==0:
            pass
        else:
            if target_B[0] or target_B[0] == i[1] or target_B[1] == i[0] or target_B[1] == i[1]:
                copy_i = copy.copy(i)
                tasks.put(copy_i)
                unequal_BBH_data[unequal_BBH_data.index(i)][2] = 0
    return

###### 13.Matching_BBH(target)

In [18]:
def Matching_BBH(target):
    "match the backward best hit"
    if target[2] == 0:
        return
    else:
        copy_target = copy.copy(target)
        Search_Equal_BBH_Data(copy_target[0])
        Search_Equal_BBH_Data(copy_target[1])
        results.put(copy_target)
        unequal_BBH_data[unequal_BBH_data.index(target)][2] =0
    for j in unequal_BBH_data:
        if j[2]==0:
            pass
        else:
            if copy_target[0] or copy_target[0] ==j[1] or copy_target[1] ==j[0] or copy_target[1]==j[1]:
                copy_j = copy.copy(j)
                print("targ_get, j = %s %s (copy_target,j)",j)
                unequal_BBH_data[unequal_BBH_data.index(j)][2] =0
                tasks.put(copy_j)
        while not tasks.empty():
            get_task = tasks.get()
            Search_Equal_BBH_Data(get_task[0])
            Search_Equal_BBH_Data(get_task[1])
            results.put(get_task)
            Search_Unequal_BBH_Data(get_task)

###### 14.Generating_Matrix_Clustering_Ortholog(element_set,bar)

In [19]:
def Generating_Matrix_Clustering_Ortholog(element_set,bar):
    """Generate the matrix of clustering ortholog """
    row_data = []
    col_data = []
    temp_results = queue.Queue()
    bar.next()
    for element in element_set:
        if row_data.count(element[0])>0:
            #if element[0] exist, returning the index in the row_data.
            row = row_data.index(element[0])
            # element[0] is data of row. ['gi|15606057|ref|NP_213434.1|', 'gi|15606057|ref|NP_213434.1|', '3823\n']
        else:
            row = len(row_data)
            row_data.append(element[0])
            if col_data.count(element[0])<1:
                #if element[0] doesn't exists, appending the element[0] to the col_data.
                col_data.append(element[0])
        if col_data.count(element[1])>0:
            col = col_data.index(element[1]) #element[1] is data of col
        else:
            col = len(col_data)
            col_data.append(element[1])
            
        temp_results.put([row,col,element[2]])
        
    score_metrix = numpy.matlib.zeros((len(row_data),len(col_data)),dtype = numpy.float)
    #Create a new matrix of a given shape (the size_results) and type, filled with zeros.
    while not temp_results.empty():
        get_temp_results = temp_results.get()
        row = get_temp_results[0]
        col = get_temp_results[1]
        score_matrix[row,col] = get_temp_results[2]
        score_matrix[col,row] = get_temp_results[2]
    if len(row_data)> 1000 and cpu-count >1:
        #The big size of Matrix(bigger than 1000 X 1000) will be computed by Parallel_Matrix_Multiplication_Using_Numpy function
        score_matrix = Parallel_MCL(score_matrix)
        
    else:
        score_matrix = MCL(score_matrix)
    Clustering(row_data,col_data, score_matrix)

###### 15.Parallel_MCL(score_matrix)

In [20]:
def Parallel_MCL(score_matrix):
    count = 0
    infinitesimal_value = 10**-10
    idempotent_matrix = numpy.matlib.ones((2,2)) #In newer Version np.ones
    while idempotent_matrix.sum() > infinitesimal_value: 
        #Greater than infinitesimal _ value
        MCL_time_start = time.time()
        pool = multiprocessing.Pool(cpu_count)
        #multiprocessing.Pool(processes=None,initializer=None,initargs=(),maxtasksperchild=None,) PARAMETER
        multipplication_results = pool.map(Parallel_Matrix_Multiplication_Using_Numpy, zip(score_matrix, repeat(score_matrix)))
        pool.close()
        pool.join()
        
        pool = multiprocessing.Pool(cpu_count) # Create a inflation_matrix (part 1)
        power_results = pool.map(Parallel_Matrix_Power_Usint_Numpy, multipplication_results)
        pool.close()
        pool.join()
        
        sum_matrix = 0
        for i in power_results:
            sum_matrix = i+ sum_matrix
        pool = multiprocessing.Pool(cpu_count) #Create a inflation_matrix (part 2)
        divide_results = pool.map(Parallel_Matrix_Power_Usint_Numpy, zip(power_results, repeat(sum_matrix)))
        pool.close()
        pool.join()
        
        for i in range(len(divide_results)):
            #Make a Combined matrix for results of Parallel_Matrix_Multiplication_Using_Numpy Function.
            if i ==0:
                score_matrix = divide_results[i]
            else:
                score_matrix = numpy.concatenate((score_matrix, divide_results[i]),axis =0)
        sum_results = 0
        for i in multiplication_results:
            sum_results += i
        idempotent_matrix = abs(numpy.sum(score_matrix) - sum_results)
        #identify whether inflation_matrix is idempotent matrix or not
        count +=1
        if count > infinite_loop:
            #It will prevent the infinite loop of MCL algorithms.
            break
        MCL_time_finish = time.time()
        if verbose:
            print("MCL time %f, count :%d, matrix size :%d *%d "%((MCL_time_finish-MCL_time_start)/60),count,score_matrix[0].size)
    return score_matrix

###### 16. MCL(score_matrix)
A^2 = A. In linear  algebra, an idempotent matrix is a matrix which, when multiplied by itself, yields itself.

In [21]:
def MCL(score_matrix):
    "This function run MCL....."
    print("MCL is Runing")
    count = 0
    infinitesimal_value = 10*-10
    idempotent_matrix = numpy.matlib.ones((2,2)) #np.ones((2,2))
    while idempotent_matrix.sum() > infinitesimal_value: # sum is 4
        MCL_time_start = time.time()
        expansion_matrix = score_matrix ** 2
        score_matrix = numpy.power(expansion_matrix, inflation_factor)
        #np.power(matrix, int)
        score_matrix_sum = score_matrix.sum(axis = 0)
        score_matrix = numpy.divide(score_matrix,score_matrix_sum) # Create a inflation_Matrix
        idempotent_matrix = abs(score_matrix - expansion_matrix)
        #identify weather inflation_matrix is idempotent matrix or not
        count +=1
        if count > infinite_loop:
            #It will prevent the infinite loop of MCL algorithm.
            break
        MCL_time_finish = time.time()
        if verbose:
            print("MCL time:%f, Count: %d , matrix size: %d *%d"%((MCL_time_finish-MCL_time_start)/60,count, score_matrix[0].size,\
                                                                 score_matrix[0].size))
            
    return score_matrix
        
        

###### 17. Clustering(row_data, col_data, score_matrix)

In [22]:
def Clustering(row_data, col_data, score_matrix):
    global cluster_count, ortholog_count
    ortholog_temp_list = []
    for i in range(len(row_data)):
        ortholog_list = []
        ortholog_sum = 0
        ortholog = queue.Queue() #It is Queue which is put the orhtolog.
        gene_id_queue = queue.Queue() # It is Queue which is put the orhtolog having changed gene ID
        for j in range(len(col_data)):
            if 0.1 <= score_matrix[i,j]:
                ortholog.put(col_data[j])
                ortholog_list.append(col_data[j])
    if ortholog.qsize() >=3: # If the ortholog queue has the element of ortholog more than2, it will be printed.
        for element in ortholog_list:
            try:
                ortholog_sum += ortholog_temp_list.index(element) + 1
            except ValueError:
                with open(Cluster_out+"_geneID_S"+str(threshold_score)+"_"+str(inflation_factor),"a") as ortholog_list_save:
                    ortholog_print = "cluster"+str(cluster_count)+":"
                    ortholog_list_save.write(ortholog_print)
                    
                    while not ortholog.empty():
                        get_ortholog = ortholog.get()
                        ortholog_list_save.write("\t"+get_ortholog)
                        try:
                            get_ortholog_split = get_ortholog.split("_") #get_ortholog --> ECO_170082288, get_ortholog.split("_")
                            gene_id_que.put(get_ortholog_split[0]+"_"+gene_id_dic[get_ortholog_split[1]])
                        except KeyError:
                            gene_id_que.put(get_ortholog) #If the gene_id_dic dont have get_ortholog, it will print the original ID(get_ortholog).
                    ortholog_list_save.write("\n")
                with open(Cluster_out +"_KO_ID_S"+str(threshold_score)+"_"+str(inflation_factor),"a") as ortholog_list_geneID_save:
                    
                        ortholog_list_geneID_print = "cluster "+ str(cluster_count)+" :"
                        ortholog_list_geneID_save.write(ortholog_list_geneID_print)
                        while not gene_id_queue.empty():
                            ortholog_list_geneID_save.write("\t"+gene_id_queue.get())
                            ortholog_count +=1
                            cluster_count +=1
                            ortholog_list_geneID_save.write("\n")
                        break
                ortholog_temp_list = operator.concat(ortholog_temp_list, ortholog_list)
                

###### 18.Parallel_Matrix_Multiplication_Using_Numpy(data)

In [23]:
def Parallel_Matrix_Multiplication_Using_Numpy(data):
    
    matrix_element, matrix  = data
    #mul = numpy.multiply(matrix_element,matrix)
    mul = matrix_element * matrix
    return 

###### 19.Parallel_Matrix_Power_Using_Numpy(matrix_element):

In [24]:
def Parallel_matrix_Power_Using_Numpy(matrix_element):
    "This Function return Metrix Power by inflation_factor"
    power_matrix_element = numpy.power(matrix_element, inflation_factor)    

###### 20.Parallel_Matrix_Divide_Using_Numpy(data)

In [25]:
def Parallel_Matrix_Divide_Using_Numpy(data):
    "data is 2 values. First is matrix_element, sum_data."
    matrix_element , sum_data = data
    return numpy.divide(matrix_element, sum_data)
    

###### 21. Read_Species_List()

In [26]:
import glob
def Read_Species_List(pr=0):
    """ If pr is 1, it will print "Species_List"  Other wise only return the Value in dic Format """ 
    read_species = os.listdir(Species) 
    selected_species_dic = {}  #list 
    backward_selected_species_dic = {} #list
    for i, species in enumerate(sorted(read_species), start=1): #
        selected_species_dic[i] = species
        backward_selected_species_dic[species] = i 
        if pr == 1 :
            print (str(i)+".", species)
        number = i
    return selected_species_dic, backward_selected_species_dic, number

###### 22. Del_File(path,file)

In [27]:
def Del_File(path,file):
    import subprocess
    
    del_file = subprocess.Popen(["del "+path+file],stdout=subprocess.PIPE, stderr = subprocess.PIPE,shell= True)
    del_file_stream = del_file.communicate()
    if not del_file_stream[1]:
        print("Done to del"+path+file)
    elif del_file_stream[1]:
        print(del_file_stream[1])

#### i developed the below Function to remove the file may be different but will check later 

In [28]:
def del_(file):
    import os
    try:
        os.remove(file)
        print("Successfully_remove",file)
    except:
        print("Check the File And Path")

###### 23.Check_File(file)

In [29]:
def Check_File(file):
    "Check the file weather exist or not"
    file_list = glob.glob(file+'*')
    
    print(file_list)
    
    if (Cluster_out+"_geneID_S"+str(threshold_score)+"_"+str(inflation_factor) or Cluster_out+"_KO_ID_S"+str(threshold_score)+"_"+str(inflation_factor)) in file_list:
        print ("Please, set other name of output.")
        sys.exit(2)

###### 24.Read_Equal_BBH(path)

In [30]:
def Read_Equal_BBH(path):
    "Read_Equal BBH by user path"
    global threshold_score
    with open(path + "_oneway_threshold_best_hit_S"+str(threshold_score),"r") as equal_RBH:
        for j in equal_RBH:
            split_data = j.split()
            split_data[2] = int(split_data[2])
            equal_BBH_data.append(split_data)
            equal_BBH_data_dic[split_data[0]] = split_data[1:]
    try:
        with open(path+"_second_oneway_threshold_best_hit_S"+str(threshold_score),"r") as second_equal_RBH:
            for j in second_equal_RBH:
                split_data = j.split()
                split_data[2] = int(split_data[2])
    except:
        pass


###### 25. Read_Unequal_BBH(path):

In [31]:
def Read_Unequal_BBH(path):
    "Read Unequal BBH path passed by User"
    with open(path+"_oneway_threshold_best_hit_S"+str(threshold_score),"r") as unequal_RBH:
        for j in unequal_RBH:
            split_data[2] = int(split_data[2])
            unequal_BBH_data.append(split_data)

####  Main Program Testing
`************* No Function below **************************`

In [32]:
# arg parse is not done here
# If parameter not Provided the below code will run

In [64]:
#if not sys.argv[1:]
# Program Not Run by Command Line
import sys
import multiprocessing

# If Program not Passed with Parameter
 # if Not verified Conflict arise for local and global Variable
print ("1. BLASTP. \n2. BLASTP using precalculated data. \n3. Clustering.\n")
mode = input(">> Select a mode or modes (1 or 2 or 1 3 or 2 3): ") 
selected_species_dic, backward_selected_species_dic, number_i = Read_Species_List(pr=1)
selected_number = input(">> Select Genomes to detect Orthologs(e.g. 1 2 3 4 5 or 1-5) : ")


if selected_number.find("-")>0:
    SN = selected_number.split("-")
    if int(SN[-1])>number_i:
        print("Your input must be less than" , number_i)
        sys.exit(2)
    else:
        user_selected_number = range(int(SN[0]),int(SN[-1])+1)
        print(user_selected_number)
        for j in user_selected_number:
            print(selected_species_dic[j],end=" ")
        print("Are Selected")
else:
    user_selected_number = sorted(set([int(read_species) for read_species in selected_number.split()]))
    if int(user_selected_number[-1])>number_i:
        print ("\nWrongInput\nInput must be less than",number_i)
        sys.exit(2)
    else:
        for j in user_selected_number:
            print(selected_species_dic[j], end =" ")

blastp_matrix = GetMatrixNumber()
#This Function only Return Matrix name as String

cpu_count = int(input("You can use %s processors. \nIf you input >2, The Program will run a parallel--"\
                      %multiprocessing.cpu_count()+"Enter the number of CPU you want to use :"))
if "3" in mode:
    inflation_factor = input("Enter the inflation factor to cluster")
    Cluster_out = input("Set the name of Clustering output :")
    

1. BLASTP. 
2. BLASTP using precalculated data. 
3. Clustering.

>> Select a mode or modes (1 or 2 or 1 3 or 2 3): 1 3
1. AAE
2. CAC
3. ECO
4. ECU
5. HIN
6. LLA
7. Log
8. SCE
9. SPO
10. SPY
11. SYN
12. TMA
13. YPE
14. chicken.faa
15. chimpanzee.faa
16. dog.faa
17. elegans.faa
18. fruitfly.faa
19. human.faa
20. mouse.faa
21. opossum.faa
22. pufferfish.faa
23. rat.faa
24. seasquirt.faa
25. zebrafish.faa
>> Select Genomes to detect Orthologs(e.g. 1 2 3 4 5 or 1-5) : 1 2 3 4 5
AAE CAC ECO ECU HIN 
1. BLOSUM45
2. BLOSUM62
3. BLOSUM82
4. Quit

Enter a matrix number: 1
You can use 12 processors. 
If you input >2, The Program will run a parallel--Enter the number of CPU you want to use :2
Enter the inflation factor to cluster1.4
Set the name of Clustering output :result_jupyternotebook


##### if Paramater Passed with Python Code

In [34]:
##elif sys.argv[1:]
##command_options = argparse_value
#genomes = command_options.genomes
#mode = command_options.mode
#cpu_count= command_options.cpu_count
#blastp_matrix = command_options.blastp_matrix
#inflation_factor = command_options.inflation_factor
#selected_species_dic,backward_selected_species_dic, number_i = Read_Species_List()
#user_selected_number = [backward_selected_species_dic[ele] for ele in genomes]
#Cluster_out = command_options.Cluster_out

In [35]:
#score_file = "score"
#save_raw_blastp_score  = 100
#import datetime

In [36]:
# These are default Valuse
Blastp = "blastp"
Blastp_data = "./blastp_data/"
#blastp_matrix = #input by user default = BLOSUM62
#cpu_count  = #user input we use 1
infinite_loop = 60
inflation_factor = 1.4
Cluster_out = "./cluster_out/"
threshold_score =0  # 5 is Passed in Server
save_raw_blastp_score = False
Score_file = "./score_file/"
Species = "./species/"
verbose = False

#infinite_loop = command_options.infinite_loop

In [65]:
#Species = command_options.Species
#Blastp = COmmand_options.Blastp
#Score_file = command_options.Score_file
#Blastp_data = command_options.Blastp_data
#save_raw_blastp_score = command_options.threshold_score
#threshold_score = command_options.threshold_score
#verbose = command_options.verbose
#infinite_loop = command_options.infinite_loop
#
if "3" in mode:
    Check_File(Cluster_out)

#Del_File(score_file, "*")
if "3" in mode:
    Log_file_name = Cluster_out+"_S"+str(threshold_score)+"_"+str(inflation_factor)+".log"
elif not "3" in mode:
    Log_file_name = "Log.txt"

with open(Log_file_name, "w") as log:
    log.write("\t\t\t"+str(datetime.datetime.now()))
    log.write("\n\t\t\tkrishdb38@gmail.com")
    log.write("\nmode")
    for i in mode:
        log.write(" "+i)
    log.write("\ngenomes:")
    for i in user_selected_number:
        log.write(selected_species_dic[i]+" ")
    log.write("\nCPU_Count:"+str(cpu_count))
    log.write("\nblastp matrix :"+blastp_matrix)
    if "3" in mode:
        log.write("\ninflation_factor: "+str(inflation_factor))
        log.write("\nCluster out:"+Cluster_out)
    log.write("\nSpecies :"+Species)
    log.write("\nBlastp: "+Blastp)
    log.write("\nScore file : "+ Score_file)
    log.write("\nBlastp_data : "+ Blastp_data)
    log.write("\nsave rawblastp score : "+ str(save_raw_blastp_score))
    log.write("\n")

[]


In [66]:
start_time_OBH = time.time()
if "1" in  mode:
    backward_best_hit_work_list = Oneway_Threshold_Best_Hit(mode)
    pool = multiprocessing.Pool(cpu_count)
    results = pool.map(Backward_Best_Hit,backward_best_hit_work_list)
    pool.close()
    pool.join()
elif "2" in mode:
    used_precalculated_data_list = []
    new_calculated_data_list = []
    precalculated_data_list = glob.glob(Blastp_data+"*oneway_threshold_best_hit_S"+str(threshold_score))
    print(precalculated_data_list)
    backward_best_hit_work_list = Oneway_Threshold_Best_Hit(mode)
    
    if not backward_best_hit_work_list ==[]:
        #If backward_best_hit_work_list is an empty list, pool instance can't finish the work.
        pool = multiprocessing.Pool(cpu_count)
        results = pool.map(Backward_Best_Hit, backward_best_hit_work_list)
        pool.close()
        pool.join()
    else:
        results = [0,0]
#Del_File("./","query*")   # !!! Later We remove these query Files
finish_time_OBH = time.time()
blastp_time_log = float(((finish_time_OBH - start_time_OBH)/60))
print("BLASTP searches + forward best Hit + backwardbest hit took %f minutes"%blastp_time_log)

with open(Log_file_name , "a") as log:
    log.write("Backward_Best_Hit took "+str(max(results))+" minutes\n")
    log.write("BLASTP + Best_Hit + backward_best_hit searches took"+ str(blastp_time_log)+" minutes\n")
if "3" in mode:
    start_time_clustering = time.time()
    #generate matrix calculate the matrix using MCL algorithm and cluster the Ortholog
    
    print("\n >>> Start MCL algorithm and Clustering ortholog <<<")
    equal_BBH_data = []
    unequal_BBH_data = []
    equal_BBH_data_dic = {}
    second_equal_BBH_data = []
    results = que.Queue()
    tasks = queue.Queue()
    cluster_count = 1
    ortholog_count = 0
    gene_id_dic = {}
    
    with open("myva=gb","r") as id_read:
        #myvba=gb is a database
        for i in id_read:
            gene_name,gene_id = i.split()
            gene_id_dic[gene_id.replace("\n","")] = gene_name # remove '\n'
            
        if "1" in mode:
            for i in user_selected_number:
                for k in user_selected_number:
                    if k<i:
                        pass
                    elif i ==k:
                        Read_Equal_BBH(Score_file+selected_species_dic[i]+"_"+selected_species_dic[k])
                    elif i !=k:
                        Read_Unequal_BBH(Score_file+selected_species_dic[i]+"_"+selected_species_dic[k])
        elif "2" in mode:
            for used_data in used_precalculated_data_list:
                first,second = used_data.split("_")
                if first == second:
                    Read_Equal_BBH(Blastp_data+used-data)
                elif first != second:
                    Read_Unequal_BBH(Blastp_data+used_data)
            for new_data in new_calculated_data_list:
                first,second = new_data.split("_")
                if first== second:
                    Read_Equal_BBH(Score_file+new_data)
                elif first != second:
                    Read_Unequal_BBH(Score_file+new_data)
        matched_BBH_data = []
        matched_BBH_element_data_set = []
        
        for unequal_RBH_element in unequal_BBH_data:
            Matching_BBH(unequal_RBH_element)
            temp_results_list = []
            if results._qsize() !=0: #return the number of results as Queue.
                while not results.empty():
                    get_results = results.get()
                    temp_results_list.append(get_results)
                matched_BBH_data.append(temp_results_list)
        #bar = bar("Processing", max= len(matched_BBH_data))
        #bar = bar("Processing", max= len(matched_BBH_data)) #Not Supported in Python 3
        #for data in matched_BBH_data:
        #    Generating_Matrix_Clustering_Ortholog(data, bar)
        #bar.finish()
        finish_time_clustering = time.time()
        mcl_time_log = float((finish_time_clustering - start_time_clustering)/60)
        remark_time_log = float((finish_time_clustering- start_time_OBH)/60)
        print("MCL algorithm and Ortholog Clustering took %.2f minutes"%remark_time_log)
        
        if "3" in mode:
            with open(Log_file_name,"a") as log:
                log.write("Ortholog Count: "+str(ortholog_count)+","+"Cluster count: "+ str(cluster_count-1)+"\n")
                log.write("MCL algorithm and Ortholog Clustering took "+str(mcl_time_log)+"minutes\n")
                log.write("xxx Program took "+str(remark_time_log)+"minutes\n")

Doing the blastp & Forward best hit searches between AAE genome and AAE genome


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
results